In [2]:
! pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.0/777.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install tensorflow
! pip install --upgrade pip


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.8 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [18]:
!pip install tqdm

In [21]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.1 MB/s eta 0:00:0000:010:01


In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tensorflow as tf
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
from textblob import Word, TextBlob

In [23]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
num_tpus_available = len(tf.config.experimental.list_physical_devices('TPU'))
print("Num TPUs Available: ", num_tpus_available)
assert num_tpus_available > 0

Num TPUs Available:  8


# Load Data

In [41]:
# Load data
train_data = pd.read_csv('/kaggle/input/amazon-comments/train_data.csv', low_memory=False)
test_data = pd.read_csv('/kaggle/input/amazon-comments/test_data.csv')
map_data = pd.read_csv('/kaggle/input/amazon-comments/title_brand.csv')

In [42]:
train_data.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,2,NaN,False,2016-11-11,A2OSUEZJIN7BI,0511189877,NaN,Chris,I have an older URC-WR7 remote and thought thi...,Cannot Learn,1478822400
1,5,NaN,True,2016-06-06,A2NETQRG6JHIG7,0511189877,NaN,Qrysta White,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,1465171200
2,4,NaN,True,2016-03-10,A12JHGROAX49G7,0511189877,NaN,Linwood,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,1457568000
3,5,NaN,True,2016-01-14,A1KV65E2TMMG6F,0511189877,NaN,Dane Williams,I got tired of the remote being on the wrong s...,Same as TWC remote,1452729600
4,5,NaN,True,2016-10-20,A280POPEWI0NSA,0594459451,NaN,Kristina H.,After purchasing cheap cords from another webs...,Good Quality Cord,1476921600


In [43]:
null_df = pd.concat([train_data.isna().sum() * 100 / len(train_data) , train_data.isna().sum()], axis=1)
null_df.columns=["percent", "num"]
null_df

,percent,num
overall,0.000000,0
vote,77.177499,647476
verified,0.000000,0
reviewTime,0.000000,0
reviewerID,0.000000,0
asin,0.000000,0
style,41.520173,348331
reviewerName,0.027058,227
reviewText,0.000000,0
summary,0.009059,76


In [44]:
test_data.isna().sum() * 100 / len(test_data) 

vote              75.335
verified           0.000
reviewTime         0.000
reviewerID         0.000
asin               0.000
style             42.480
reviewerName       0.020
reviewText         0.000
summary            0.010
unixReviewTime     0.000
dtype: float64

In [185]:
df = train_data.copy()

In [184]:
def clean_text(text):
    # Remove non-alphanumeric characters and extra whitespace
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert multiple whitespace characters to a single space
    text = re.sub(r'\s+', ' ', text)
    # Convert the text to lowercase
    text = text.lower()
    return text

In [203]:
df.head().iterrows()

<generator object DataFrame.iterrows at 0x7b6bc93b3350>

In [210]:
def text_preprocessing(dataframe, dependent_var):
    for i in tqdm(range(7)):
        if i == 0:
            # Normalizing Case Folding - Uppercase to Lowercase
            dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
        if i == 1:
            # Removing Punctuation
            dataframe[dependent_var] = dataframe[dependent_var].str.replace('[^\w\s]','')
        if i == 2:
            # Removing Numbers
            dataframe[dependent_var] = dataframe[dependent_var].str.replace('\d','')
        if i == 3:
            # Convert multiple whitespace characters to a single space
            dataframe[dependent_var] = dataframe[dependent_var].str.replace('\s+',' ')
        if i == 4:
            # StopWords
            sw = stopwords.words('english')
            dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
        if i == 5:
            # Remove Rare Words
            temp_df = pd.Series(' '.join(dataframe[dependent_var]).split()).value_counts()
            drops = temp_df[temp_df <= 1]
            dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join(x for x in str(x).split() if x not in drops))
        if i == 6:
            # Lemmatize
            dataframe[dependent_var] = dataframe[dependent_var].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

    return dataframe

In [211]:
df['reviewText'][8]

"it works! i was skeptical about ordering it because the ratings weren't great, but short of buying a new nook or replacing it with a kindle, it was the best short-term option. the original plug-in part made a sizzling sound one night and stopped working. my husband tried to plug the usb end into one of about 20 usb to plug-in wall chargers that we have around the house and none of them worked. we're assuming there must be some proprietary thing with the nook usb end that won't let you use generic wall chargers which really sucked because he couldn't use his nook for weeks. if you are contemplating purchasing a nook, it may be something to take into consideration."

In [212]:
df = text_preprocessing(df, "reviewText")

 86%|████████▌ | 6/7 [05:50<00:58, 58.49s/it]


NameError: name 'Word' is not defined

In [ ]:
df['reviewText'][8]

In [ ]:
df.loc[:, 'reviewText'] = 

In [162]:
df["Sentiment"] = df["overall"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})

In [163]:
df['review_lengths'] = df['reviewText'].apply(len)

In [164]:
df['summary'] = df['summary'].fillna('')

In [165]:
df['summary_lengths'] = df['summary'].apply(len)

In [166]:
df['review_lengths'].max()

29146

In [167]:
df['summary_lengths'].max()

353

In [168]:
df['overall'].value_counts() / len(df)

overall
5    0.550078
4    0.186561
1    0.098874
3    0.096835
2    0.067652
Name: count, dtype: float64

In [169]:
df[df['review_lengths'] < 600]['overall'].value_counts() / len(df)

overall
5    0.395558
4    0.119675
1    0.068736
3    0.063468
2    0.044372
Name: count, dtype: float64

## Replace reviewText with their summary where len(reviewText) > 800

In [170]:
(df[df['review_lengths'] > 600]['summary'] == '').sum()

21

In [171]:
large_review_indexes = df[df['review_lengths'] > 600].index 

In [172]:
df.loc[large_review_indexes, 'reviewText'] = df.loc[large_review_indexes, 'summary']

In [173]:
df['review_lengths'] = df['reviewText'].apply(len)

In [174]:
df[df['review_lengths'] > 600]['overall'].value_counts() / len(df)

Series([], Name: count, dtype: float64)

## Deplace where len(reviewText) == 0

In [175]:
review_0_indexes = df[df['review_lengths'] == 0].index 
review_0_indexes

Index([  8659,  49919,  88710,  90185,  98294, 107058, 220081, 285391, 331359,
       359723, 447136, 511307, 524923, 571494, 628925, 647542, 672415, 690173,
       725943, 780052, 824158],
      dtype='int64')

In [176]:
df = df.drop(review_0_indexes)

In [177]:
(df['review_lengths'] == 0).sum()

0

## Drop extra columns

In [178]:
df_train = df[["reviewText", "Sentiment"]]

In [182]:
print('❤')

❤


In [179]:
df_train.head()

,reviewText,Sentiment
0,I have an older URC-WR7 remote and thought thi...,0
1,First time I've EVER had a remote that needed ...,1
2,Got them and only 2 of them worked. company ca...,1
3,I got tired of the remote being on the wrong s...,1
4,After purchasing cheap cords from another webs...,1


## Sampling

In [180]:
df_train['Sentiment'].value_counts()

Sentiment
1    699221
0    139702
Name: count, dtype: int64